In [83]:
import pandas as pd
import numpy as np
import pyarrow
import requests
from bs4 import BeautifulSoup
import re

In [84]:

'''
masterdf = pd.DataFrame(columns = ["all_text", "url"])

for i in range(1, 10320):
    
    try:
        # Specify the URL you want to scrape
        url = 'https://nkp.gov.np/full_detail/'+str(i)

        # Send an HTTP GET request to the URL
        response = requests.get(url, verify = False)

        # encoding
        response.encoding = 'utf-8'

        # Get the HTML content from the response
        html_content = response.text

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Optionally, print the formatted HTML
        faisala_div = soup.find("div", id="faisala_detail ")
        #print(faisala_div)
        df = pd.DataFrame({"all_text": [str(faisala_div)], "url": [url]})
        masterdf = pd.concat([masterdf, df])
    
    except:
        pass

'''


'\nmasterdf = pd.DataFrame(columns = ["all_text", "url"])\n\nfor i in range(1, 10320):\n    \n    try:\n        # Specify the URL you want to scrape\n        url = \'https://nkp.gov.np/full_detail/\'+str(i)\n\n        # Send an HTTP GET request to the URL\n        response = requests.get(url, verify = False)\n\n        # encoding\n        response.encoding = \'utf-8\'\n\n        # Get the HTML content from the response\n        html_content = response.text\n\n        # Parse the HTML content using BeautifulSoup\n        soup = BeautifulSoup(html_content, \'html.parser\')\n\n        # Optionally, print the formatted HTML\n        faisala_div = soup.find("div", id="faisala_detail ")\n        #print(faisala_div)\n        df = pd.DataFrame({"all_text": [str(faisala_div)], "url": [url]})\n        masterdf = pd.concat([masterdf, df])\n    \n    except:\n        pass\n\n'

In [81]:
# Load the Parquet file into a DataFrame
df = pd.read_parquet("C:\\Users\\Giri\\Desktop\\Sankhya\\Legal\\legaldata.parquet")

In [66]:
# Function to extract only Devanagari text
def extract_devanagari(html_text):
    soup = BeautifulSoup(html_text, "html.parser")  # Remove HTML tags
    text = soup.get_text()
    devnagari_text = re.findall(r'[\u0900-\u097F\s।,]+', text)  # Keep only Devanagari characters
    return ''.join(devnagari_text).strip()  # Join and remove leading/trailing spaces

In [67]:
# Apply function to the DataFrame column
masterdf['clean_text'] = masterdf['all_text'].apply(extract_devanagari)

In [72]:
def extract_judges(text):
    if pd.isna(text):  # Handle missing values
        return None
    
    pattern = r'माननीय न्यायाधीश [श्री]*\s*([\u0900-\u097F\s]+)'
    judges = re.findall(pattern, text)
    
    return [name.strip() for name in judges] if judges else None  # Return cleaned list or None


In [73]:
masterdf['judges'] = masterdf['clean_text'].apply(extract_judges)

In [75]:
masterdf["clean_text"][2]

'सर्वोच्च अदालत, विशेष इजलास\nमाननीय न्यायाधीश श्री कल्याण श्रेष्ठ\nमाननीय न्यायाधीश श्री सुशीला कार्की\nमाननीय न्यायाधीश श्री बैद्यनाथ उपाध्याय\nमाननीय न्यायाधीश श्री तर्कराज भट्ट\nमाननीय न्यायाधीश श्री ज्ञानेन्द्रबहादुर कार्की\n\xa0\nआदेश मिति  २०७०।१२।२०।५\n०६९००११\n\xa0\nविषयः उत्प्रेषण, परमादेश ।\n\xa0\nनिवेदक  प्रहरी प्रधान कार्यालय, नक्सालमा प्रहरी अतिरिक्त महानिरीक्षक पदमा कार्यरत् मदनबहादुर खड्कासमेत\nविरूद्ध\nविपक्षी  नेपाल सरकार, प्रधानमन्त्री तथा मन्त्रिपरिषद्को कार्यालय, सिँहदरबार, काठमाडौंसमेत\n\xa0\n\xa0 शान्ति सुरक्षासँग सम्बन्धित संवेदनशील अङ्गको सेवाको प्रशासन संयन्त्रलाई राजनीतिक वा प्रशासनिक स्वविवेकको अधीनमा राखी तदर्थवादी र मौकापरस्त ढङ्गले सञ्चालन गर्ने सोच मात्रै पनि अवाञ्छनीय हुने । \n\xa0 प्रहरी सेवामा सेवा प्रवेशदेखि वृत्ति विकास र सेवाबाट अवकाशसम्मका कुराहरू स्पष्टरूपमा सुपरिभाषित, सुनिश्चित र निर्दिष्ट बनाई राख्न र अन्योलको अवस्था हटाई सुस्पष्ट बनाउन जो चाहिने प्रयास अविलम्ब गर्नुपर्ने ।\nप्रकरण नं१६\n\xa0 प्रहरी ऐन, २०१२\xa0 जुन परिवेशमा आएको थियो, सो सन्द

In [76]:
# Function to extract the first Nepali date (YYYY।MM।DD)
def extract_nepali_date(text):
    if pd.isna(text):  # Handle missing values
        return None
    
    pattern = r'\d{4}।\d{1,2}।\d{1,2}'  # Matches YYYY।MM।DD
    match = re.search(pattern, text)  # Find first match
    
    return match.group() if match else None  # Return date or None

In [78]:
masterdf['decision_date'] = masterdf['clean_text'].apply(extract_nepali_date)

In [80]:
masterdf.tail()

,index,all_text,url,clean_text,judges,decision_date
5496,0,"<div id=""faisala_detail "">\n<img class=""waterm...",https://nkp.gov.np/full_detail/9530,"सर्वोच्च अदालत, संयुक्त इजलास\nमाननीय न्यायाधी...",[प्रकाशमान सिंह राउत\nमाननीय न्यायाधीश श्री हर...,२०७६।८।१३
5497,0,"<div id=""faisala_detail "">\n<img class=""waterm...",https://nkp.gov.np/full_detail/9531,"सर्वोच्च अदालत, संयुक्त इजलास\nमाननीय न्यायाधी...",[सपना प्रधान मल्ल\nमाननीय न्यायाधीश श्री बमकुम...,२०७६।१०।१७
5498,0,"<div id=""faisala_detail "">\n<img class=""waterm...",https://nkp.gov.np/full_detail/9532,"सर्वोच्च अदालत, संयुक्त इजलास\nमाननीय न्यायाधी...",[सपना प्रधान मल्ल\nमाननीय न्यायाधीश श्री सुष्म...,२०७७।३।५
5499,0,"<div id=""faisala_detail "">\n<img class=""waterm...",https://nkp.gov.np/full_detail/9533,"सर्वोच्च अदालत, संयुक्त इजलास\nमाननीय न्यायाधी...",[सपना प्रधान मल्ल\nमाननीय न्यायाधीश डा श्री मन...,२०७७।१।१४
5500,0,"<div id=""faisala_detail "">\n<img class=""waterm...",https://nkp.gov.np/full_detail/9534,"सर्वोच्च अदालत, संयुक्त इजलास\nमाननीय न्यायाधी...",[तेजबहादुर केसी\nमाननीय न्यायाधीश श्री पुरूषोत...,२०७४।९।२०
